# Brat Tags Data Analysis

## IDs in Brat 

- T: text-bound annotation
- R: relation
- E: event
- A: attribute
- M: modification (alias for attribute, for backward compatibility)
- N: normalization [new in v1.3]

<br> <br>

### Annotation ID conventions
All annotations IDs consist of a single upper-case character identifying the annotation type and a number. The initial ID characters relate to annotation types as follows:

<br>

#### Entity annotations
Each entity annotation has a unique ID and is defined by type (e.g. Person or Organization) and the span of characters containing the entity mention (represented as a "start end" offset pair).

| ID 	| Type And Span      	| Text     	|
|----	|--------------------	|----------	|
| T1 	| Organization 0 4   	| Sony     	|
| T3 	| Organization 33 41 	| Ericsson 	|
| T3 	| Country 75 81      	| Sweden   	|

<br> 

#### Attribute and modification annotations
Attribute annotations are binary or multi-valued "flags" that specify further aspects of other annotations. Attributes have a unique ID and are defined by reference to the ID of the annotation that the attribute marks and the attribute value.

| ID 	| Type & Entity ID  	|
|----	|-------------------	|
| A1 	| Negation T1       	|
| A2 	| Confidence T2     	|

<br>

#### Relation annotations
Binary relations have a unique ID and are defined by their type (e.g. Origin, Part-of) and their arguments.

| ID 	| Type and Args          	|
|----	|------------------------	|
| R1 	| Origin Arg1:T3 Arg2:T4 	|



## First Iteration

<br>


- Hand-picked: These tweets were handpicked if they were good examples of true public services reports or if the tweet seemed to not be a report but it really was one (confusing tweets.

- Sampled: Random sample of tweets was drawn from the original dataframe. The sampling process was done apart. 

<br>

- Hand-picked


| id_parsed 	|          annotation_parsed 	| Count 	|
|----------:	|---------------------------:	|------:	|
| A         	| without-service            	| 63    	|
|           	| location                   	| 38    	|
|           	| duration                   	| 16    	|
|           	| time                       	| 10    	|
|           	| fake-information           	| 2     	|
|           	| with-service               	| 2     	|
|           	| reason                     	| 1     	|
| T         	| circumstantial-information 	| 65    	|
|           	| social-report              	| 39    	|
|           	| electricity                	| 33    	|
|           	| gasoline                   	| 25    	|
|           	| water                      	| 6     	|
|           	| gas                        	| 4     	|

<br>

- Sampled

| id_parsed 	|          annotation_parsed 	| Freq 	|
|----------:	|---------------------------:	|-----:	|
| A         	| without-service            	| 24   	|
|           	| location                   	| 24   	|
|           	| time                       	| 10   	|
|           	| utility-company            	| 5    	|
|           	| duration                   	| 4    	|
|           	| fake-information           	| 4    	|
|           	| politician                 	| 4    	|
|           	| reason                     	| 3    	|
|           	| news-company               	| 3    	|
|           	| with-service               	| 2    	|
|           	| other                      	| 1    	|
| T         	| circumstantial-information 	| 41   	|
|           	| electricity                	| 26   	|
|           	| social-report              	| 25   	|
|           	| twitter-account            	| 12   	|
|           	| gasoline                   	| 3    	|
|           	| water                      	| 1    	|
|           	| water                      	| 1    	|

In [ ]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_colwidth', None)



### Read Data
original_df = pd.read_csv('../data/processed/brat/balanced_dataset_brat.ann', sep = '\t',header = None)

## Deprecated path
# ../brat-v1.3_Crunchy_Frog/data/first-iter/balanced_dataset_brat.ann

# Rename coumns 
original_df.columns = ['id', 'annotation', 'text']

# Remove the ID numbers to know if it's an entity (T) or Attribute (A)
original_df['id_parsed'] = original_df.id.str.replace('\d', '')

# Remove text span and IDs (T & A) from column. This columns has the name of the attributes and etitites 
original_df['annotation_parsed'] = original_df.annotation.str.replace('[\dTA]', '')


# Remove Relation tags
# Change Relation Id to Null
original_df.id_parsed.replace('R', np.nan, inplace= True)

# Remove nulls
original_df.dropna(subset=['id_parsed'], inplace= True)

# Group by id_parsed, annotation parsed and count results
df = original_df[['id_parsed', 'annotation_parsed']].groupby(['id_parsed', 'annotation_parsed'], sort = True).agg({'annotation_parsed':['count']}).copy()

# After the group by there's multi-index columns. We rename the columns to have the level that we want (count)
df.columns = df.columns.levels[1]

# sort_values by index. Here the trick is to also use sort_index!!
df.sort_values('count', ascending=False)\
    .sort_index(level=[0], ascending=[True])

## Sampled

The following data was randomly sampled with the helper function ``` data_sampler.py ``` 

- Random_state: 58 
- Sample: 30 

` python data_sampler.py 58 30 brat-v1.3_Crunchy_Frog/data/first-iter/sampled_58_30.txt `


In [ ]:
complete_df = pd.read_csv('../data/raw/tweets/tagging-set-original_for_jupyter_tagging.csv')
ann = '../data/processed/brat/balanced_dataset_brat.ann'
txt = '../data/processed/brat/balanced_dataset_brat.txt'
# pd.DataFrame(complete_df.sample(30, random_state = 9).full_text)

test_balance = pd.read_csv(ann, sep = '\t')
## Deprecated path
# ../brat-v1.3_Crunchy_Frog/data/first-iter/balanced_dataset_brat.ann

# Rename coumns 
test_balance.columns = ['id', 'annotation', 'text']

# Remove the ID numbers to know if it's an entity (T) or Attribute (A)
test_balance['id_parsed'] = test_balance.id.str.replace('\d', '')

# Remove text span and IDs (T & A) from column. This columns has the name of the attributes and etitites 
test_balance['annotation_parsed'] = test_balance.annotation.str.replace('[\dTA]', '')


# Remove Relation tags
# Change Relation Id to Null
test_balance.id_parsed.replace('R', np.nan, inplace= True)

# Remove nulls
test_balance.dropna(subset=['id_parsed'], inplace= True)

# Group by id_parsed, annotation parsed and count results
df = test_balance[['id_parsed', 'annotation_parsed']].groupby(['id_parsed', 'annotation_parsed'], sort = True).agg({'annotation_parsed':['count']}).copy()

# After the group by there's multi-index columns. We rename the columns to have the level that we want (count)
df.columns = df.columns.levels[1]

# sort_values by index. Here the trick is to also use sort_index!!
df.sort_values('count', ascending=False)\
    .sort_index(level=[0], ascending=[True])


### Reading Data to input into baseline model

Strategy:

<br>

- Volarme atributos y relations. 
- Picar dataframe en espacios, anotaicion, comienzo de span y final de span
- Hacer Sort del Dataframe por la segunda clumna (comienzo de span)
- Cargar el archivo original de texto, La primera columna es la posicion en el documento de inicio de la linea
- Hacer join con el inicio de la linea. Schema - inicio de la linea y tweet. 

<br>

Roadblock

- The problem with this approach is that the text file's lines do not match with the annotation file's span. This is due that the span is for the tag, not necessarily the first word of the tweet is going to be tagged. The same is for the last tag's span vs the end of the string. 


#### Helper Function

This helper function reads brat's annotation file (.ann) and parses to a dataframe with the following schema: 


| Column            	| Dtype  	|
|-------------------	|--------	|
| id                	| object 	|
| annotation        	| object 	|
| text              	| object 	|
| id_parsed         	| object 	|
| annotation_parsed 	| object 	|

In [ ]:
import pandas as pd
def annotation_parser(dir_ann_file, print_grouped_annotations = False):
    '''
        Helper function to parse Brat's annotation file (.ann). 
            Returns a dataframe with the following schema: 
                | id                	| object 	|
                | annotation        	| object 	|
                | text              	| object 	|
                | id_parsed         	| object 	|
                | annotation_parsed 	| object 	|
    --------------------------------------------------------------
    
    Params:
        
        dir_ann_file: String, Default = None. 
            Path to ann file.
        
        print_grouped_annotation: Bool. Default = False. 
            Prints count of Entities and Attributes.
    '''
    
    ### Read Data
    to_parse_df = pd.read_csv(dir_ann_file, sep = '\t',header = None)

    # Rename coumns 
    to_parse_df.columns = ['id', 'annotation', 'text']

    # Remove the ID numbers to know if it's an entity (T) or Attribute (A)
    to_parse_df['id_parsed'] = to_parse_df.id.str.replace('\d', '')

    # Remove text span and IDs (T & A) from column. This columns has the name of the attributes and etitites 
    to_parse_df['annotation_parsed'] = to_parse_df.annotation.str.replace('[\dTA]', '')


    # Remove Relation tags
    # Change Relation Id to Null
    to_parse_df.id_parsed.replace('R', np.nan, inplace= True)

    # Remove nulls
    to_parse_df.dropna(subset=['id_parsed'], inplace= True)

    # Group by id_parsed, annotation parsed and count results
    df = to_parse_df[['id_parsed', 'annotation_parsed']].groupby(['id_parsed', 'annotation_parsed'], sort = True).agg({'annotation_parsed':['count']}).copy()

    # After the group by there's multi-index columns. We rename the columns to have the level that we want (count)
    df.columns = df.columns.levels[1]

    # sort_values by index. Here the trick is to also use sort_index!!
    
    if print_grouped_annotations == True:
    
        print(df.sort_values('count', ascending=False)\
            .sort_index(level=[0], ascending=[True]))
    else:
        pass
    
    return to_parse_df

### Read Annotations

<br>

- Used annotation_parser helper function to read the data. 
- Subset only Entities for simplicity sake. 
- Create span columns and store it into data frame split_ann

In [ ]:

# Read sampled data
sampled_ann = annotation_parser(ann) # 308 rows 

# # Deprecated path 
# ../brat-v1.3_Crunchy_Frog/data/first-iter/balanced_dataset_brat.ann

# Subset Entities and rewrite dataframe
sampled_ann = sampled_ann[sampled_ann.id_parsed == 'T'] # 109 rows

# Create span columns. Split by space.
split_ann = sampled_ann.annotation.str.split(' ', expand = True)

# Rename Columns
split_ann.columns = ['Entities', 'first_char', 'last_char'] # It's already sorted by first_char ascending

# Create new columns with each annotation's text.
split_ann['text'] = sampled_ann.loc[sampled_ann.id_parsed == 'T', 'text']

split_ann.first_char = split_ann.first_char.astype(int)
split_ann.last_char = split_ann.last_char.astype(int)

split_ann.info()

### Read Text and Merge with Annotation File

<br>

Approach

- The base strategy is to find the starting position of each new line and join it with the the first_char column in the annotation table. 

<br>

Roadblock

- The problem with this approach is that the text file's lines do not match with the annotation file's span. This is due that the span is for the tag, not necessarily the first word of the tweet is going to be tagged. The same is for the last tag's span vs the end of the string. 

<br>

To do:

- I am thinking on using fuzzy string matching to match `split_ann` text column with the original text data frame. 

In [ ]:
# Import Regex Module
import re

# Set max column width to none in order to read all the tweet's text. 
pd.options.display.max_colwidth = None

# Read text
text = pd.read_csv(txt, header=None )

## Deprecated path
# ../brat-v1.3_Crunchy_Frog/data/first-iter/balanced_dataset_brat.txt

# Rename column to tweets
text.columns = ['tweets']

# Convert tweets to string
text_string = text.tweets.to_string( header = False)

# Remove extra spaces and replace them with just 1 space.
text_string = re.sub(' +', ' ', text_string)

#last_char = [pos for pos, char in enumerate(text_string) if char == '\n']

# last_char

Attempt to solving the roadblock

In [ ]:
# read the txt as a string file
with open(txt) as f:
    # only replace the break lines
    REPLACE_br = lambda s: s.replace("\n","\n")
    lines = map( REPLACE_br, f.readlines() )
    
    # save number line, length of the text and text without break lines: /n
    # assuming one line corresponds to a single tweet
    tuple_tweets = [(len(l), l) for l in lines if len(l) > 0]

    start, end, text_ = list(), list(), list()
    new_start = 0
    for ttw in tuple_tweets:
        # adds the length of the tweet
        start.append(new_start)
        # finds the location of the last character of the tweet
        end.append(new_start + ttw[0] -1 )
        text_.append(ttw[1])
        
        # gets the starting position of the next tweet
        new_start = new_start + ttw[0]

    text_df = pd.DataFrame({
            "first_char": start,
            "last_char": end,
            "text": text_
            })
# Text - Fix de Min!!
text_df.head(10)

In [ ]:
# Annotations DF to test 

split_ann = split_ann.sort_values('first_char').copy()
split_ann.head()

### First attempt - Merging Text and Labels 

<br>

This approach doesn't take into consideration tags that were made in the middle of the tweet. 

In [ ]:
## This didn't work
# pd.merge(df, split_ann, on = ['last_char'] ).info()


text = [] # Load the matched text into a list 
ann = [] # Load the matched labels into a list

# For each text_df rows and for each split_ann rows
for index_txt, row_txt in text_df.iterrows():
    
    for index_ann, row_ann in split_ann.iterrows():
    
        # If first char of annotations and text match 
            # OR
        # If last_char of annotations matches text 
        if row_ann[1] == row_txt[0] or row_ann[2] == row_txt[1] :
            ann.append(row_ann[0])
            text.append(row_txt[2])
        else:
            next


txt_ann_df = pd.DataFrame({
                    'text':text, 
                    'ann':ann
                    })



In [ ]:
### Merging the txt with annotations

# Create Final Dataframe
txt_ann_df_concat = pd.concat([txt_ann_df, # Final Dataframe
           pd.get_dummies(txt_ann_df.ann)] # Getting dummies of all annotations and append both Dfs
          , axis = 1)

# Group by each tweet's text. The tweets are duplicated from last step 
    # (if they matched both in first_char and last_char)

    ## Tags are missing in this approach
# txt_ann_df_concat.drop(['ann'], axis = 1).groupby('text', as_index = False).sum() #drop annotation column

### Second Attempt - Merging Text and Labels

This approach takes into consideration the complete span of the text. 

It subsets all the tags that are within the beginning of the tweet and the end of the tweet. A new column "multi_id" is created in order to match both dataframes.

In [ ]:

# Initialize column multi_id
split_ann['multi_id'] = 0

# For each row of original text 
for i in range(text_df.shape[0]):
    
            # Match first character position of the annotation greater or equal to the first_char of the tweet
            # Match last character position of the annotation less than the last character of the complete tweet
            # This will subset the annotations on the complete span of the tweet.
    idx = split_ann[(split_ann.first_char >= text_df.loc[i,'first_char']) &\
              (split_ann.first_char < text_df.loc[i,'last_char'])].index   

#     Create common value for each dataframe. 
    split_ann.loc[idx, 'multi_id'] = i 
    text_df.loc[i, 'multi_id'] = i


# Merge both dataframes and select the entities and the tweets' complete text
merged = pd.merge(split_ann, text_df, on='multi_id').loc[:, ['Entities', 'text_y']]


# Create dummy variables from entities (these are the tags)
dummies = pd.get_dummies(merged.Entities)

# Group by the complete text
pd.concat([merged['text_y'], dummies], axis = 1).groupby('text_y').sum().reset_index().head()

## Helper function implementation

The output of this helper function is intended to serve as the input for the baseline machine learning model. 

In [ ]:
# Append root folder
import sys
sys.path.append('../')

# Import libraries
import numpy as np
import pandas as pd
import sys
from src.c4v.data.data_loader import BratDataLoader

# Instantiate class with one dataset
loaded_data = BratDataLoader(['../data/processed/brat/sampled_58_30'], binary=True)

## Get parsed dataframe with text and one-hot encoded responses
loaded_data.get_parsed_df()

# Create training and test sets
loaded_data.preprocess()

## Print training and test sets
print(
f'''
Training set shape:
X_train: {loaded_data.X_train.shape}
y_train: {loaded_data.y_train.shape}

Testing set shape:
X_test: {loaded_data.X_test.shape}
y_test: {loaded_data.y_test.shape}
''')